# Setup

In [1]:
import json
import requests
from datetime import datetime as dt
from collections import defaultdict
import pandas as pd

In [2]:
#https://pypi.org/project/gamma-viewer/
from gamma_viewer import GammaViewer
from IPython.display import display, Markdown

In [3]:
def printjson(j):
    print(json.dumps(j,indent=4))
def print_json(j):
    printjson(j)

In [4]:
def post(name,url,message,params=None):
    if params is None:
        response = requests.post(url,json=message)
    else:
        response = requests.post(url,json=message,params=params)
    if not response.status_code == 200:
        print(name, 'error:',response.status_code)
        print(response.json())
        return {}
    return response.json()

def automat(db,message):
    automat_url = f'https://automat.renci.org/{db}/query'
    response = requests.post(automat_url,json=message)
    print(response.status_code)
    return response.json()

def strider(message):
    url = 'https://strider.renci.org/query?log_level=DEBUG'
    strider_answer = post(strider,url,message)
    return strider_answer

def aragorn(message, coalesce_type='xnone'):
    if coalesce_type == 'xnone':
        answer = post('aragorn','https://aragorn.renci.org/query',message)
    else:
        answer = post('aragorn','https://aragorn.renci.org/query',message, params={'answer_coalesce_type':coalesce_type})
    return answer

##

def bte(message):
    url = 'https://api.bte.ncats.io/v1/query'
    return post(strider,url,message)
 
def striderandfriends(message):
    strider_answer = strider(message)    
    coalesced_answer = post('coalesce','https://answercoalesce.renci.org/coalesce/graph',{'message':strider_answer})
    omni_answer = post('omnicorp','https://aragorn-ranker.renci.org/omnicorp_overlay',{'message': coalesced_answer})
    weighted_answer = post('weight','https://aragorn-ranker.renci.org/weight_correctness',{'message': omni_answer})
    scored_answer = post('score','https://aragorn-ranker.renci.org/score',{'message': weighted_answer})
    return scored_answer

In [5]:
def retrieve_ars_results(mid):
    message_url = f'https://ars.transltr.io/ars/api/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    results = {}
    for child in j['children']:
        if child['actor']['agent'] in ['ara-aragorn', 'ara-aragorn-exp']:
            childmessage_id = child['message']
            child_url = f'https://ars.transltr.io/ars/api/messages/{childmessage_id}'
            child_response = requests.get(child_url).json()
            try:
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except:
                nresults=0
            print( child['status'], child['actor']['agent'],nresults )
    return results

In [6]:
def get_provenance(message):
    """Given a message with results, find the source of the edges"""
    prov = defaultdict(lambda: defaultdict(int)) # {qedge->{source->count}}
    results = message['message']['results']
    kg = message['message']['knowledge_graph']['edges']
    edge_bindings = [ r['edge_bindings'] for r in results ]
    for bindings in edge_bindings:
        for qg_e, kg_l in bindings.items():
            for kg_e in kg_l:
                for att in kg[kg_e['id']]['attributes']:
                    if att['name'] == 'provenance':
                        source = att['value']
                        prov[qg_e][source]+=1
    qg_edges = []
    sources = []
    counts = []
    for qg_e in prov:
        for source in prov[qg_e]:
            qg_edges.append(qg_e)
            sources.append(source)
            counts.append(prov[qg_e][source])
    prov_table = pd.DataFrame({"QG Edge":qg_edges, "Source":sources, "Count":counts})
    return prov_table

## Query Specific

In [7]:
standup_json='standup_1.json'

In [8]:
with open(standup_json,'r') as jsonfile:
    standup_info = json.load(jsonfile)

In [9]:
display(Markdown(f"# {standup_info['Query Title']}"))
display(Markdown(f"{standup_info['Query Description']}"))
print(f'Github Issue: {standup_info["github_issue"]}')

# 1. Gene to Pathway

What pathways are associated with CDK2 (NCBIGene:1017)

Github Issue: https://github.com/NCATSTranslator/testing/issues/9


The query as run through the ARS:

In [10]:
query = json.loads(requests.get(standup_info['query_location']).content)
printjson(query)

{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "id": "NCBIGENE:1017",
                    "category": "biolink:Gene"
                },
                "n1": {
                    "category": "biolink:Pathway"
                }
            },
            "edges": {
                "e01": {
                    "subject": "n0",
                    "object": "n1"
                }
            }
        }
    }
}


## ARS Assessment

In [11]:
ARS_Responses = [(dt.strptime(x['ARS_result_date'],'%Y-%m-%d'),x['ARS_result_id']) for x in standup_info['ARS_Results']]
ARS_Responses.sort()

In [12]:
for ars_date, ars_id in ARS_Responses:
    display(Markdown(f'### {ars_date}'))
    _ = retrieve_ars_results(ars_id)
    print(f'https://arax.ncats.io/?source=ARS&id={ars_id}')

### 2021-03-09 00:00:00

Done ara-aragorn 15
Error ara-aragorn-exp 0
https://arax.ncats.io/?source=ARS&id=5e3f38b6-2b23-479c-b04e-81fce973ca52


### 2021-03-10 00:00:00

Done ara-aragorn 0
Done ara-aragorn-exp 0
https://arax.ncats.io/?source=ARS&id=4525e749-aef6-4025-9c54-b90cfd064305


### 2021-03-18 00:00:00

Done ara-aragorn 106
Done ara-aragorn-exp 0
https://arax.ncats.io/?source=ARS&id=56085e63-c878-4634-8a70-69810f858e1e


## Strider Direct

In [13]:
start = dt.now()
strider_result = strider(query)
end = dt.now()
print(f"Strider produced {len(strider_result['message']['results'])} results in {end-start}.")

Strider produced 232 results in 0:00:48.600080.


In [14]:
prov = get_provenance(strider_result)
display(prov)

,QG Edge,Source,Count
0,e01,https://api.bte.ncats.io/v1/query,232


In [15]:
view = GammaViewer(props={"data":strider_result})
display(view)

### Strider Assessment

We return lots of edges, but they are all from BTE.  I can't easily tell but I'd like to know the underlying source.  If it's a curated source, then that's fine, but I'd like to have a curated source.   Why are we not getting results from SRI KG?

## ARAGORN 

In [16]:
start = dt.now()
aragorn_result = aragorn(query)
end = dt.now()
print(f"ARAGORN produced {len(aragorn_result['message']['results'])} results in {end-start}.")

ARAGORN produced 0 results in 0:00:04.458484.


In [17]:
print_json(aragorn_result)

{
    "message": {
        "query_graph": {
            "nodes": {
                "n1": {
                    "category": [
                        "biolink:Pathway"
                    ],
                    "is_set": false
                },
                "n0": {
                    "id": [
                        "NCBIGENE:1017"
                    ],
                    "category": [
                        "biolink:Gene"
                    ],
                    "is_set": false
                }
            },
            "edges": {
                "e01": {
                    "subject": "n0",
                    "object": "n1"
                }
            }
        },
        "knowledge_graph": {
            "nodes": {
                "BIOCARTA:rbpathway": {
                    "category": [
                        "biolink:Pathway"
                    ],
                    "name": "BIOCARTA:rbpathway",
                    "attributes": [
                        {
         

In [19]:
view = GammaViewer(props={"data":aragorn_result})
display(view)

### ARAGORN Assessment

How did we do?

We for some reason are returning no results.  We are however returning a knowledge graph.  We need to return the original results, at least if there is no coalesce, and probably no matter what https://github.com/ranking-agent/aragorn/issues/7. 

Additionally, we are getting no AC results, because we don't have pathways in the RK graph, which is what the enrichment is based on. https://github.com/ranking-agent/AnswerCoalesce/issues/42